# 📌 Projet Deep Learning : Prédiction du prix d'un véhicule & Génération d'images
Dans ce notebook, nous allons :
- Préparer et nettoyer les données.
- Entraîner un modèle de deep learning amélioré.
- Exporter le modèle pour une utilisation future.
- Générer des images de voitures avec Stable Diffusion.

In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from diffusers import StableDiffusionPipeline
import shutil
from zipfile import ZipFile

print('Librairies chargées ✅')

ModuleNotFoundError: No module named 'diffusers'

Chargement du dataset depuis google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get file from google drive
file_path = './vehicles.csv'
destination = os.getcwd()
zip_file_path = './Voitures.zip'
source = 'drive/MyDrive/Shared/Voitures.zip'
if not os.path.exists(zip_file_path):
  shutil.copy2(source, destination)
# Unzip file

if not os.path.exists(file_path):
  with ZipFile(zip_file_path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(path=".")

## 1️⃣ Chargement des données & Prétraitement

In [ ]:
# Charger les données

df = pd.read_csv(file_path, usecols=['price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'size',
                                     'type', 'paint_color', 'state', 'description'], engine='python')
df.dropna(inplace=True)
df.head()

,price,year,manufacturer,model,condition,cylinders,size,type,paint_color,description,state
31,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,full-size,truck,black,2013 F-150 XLT V6 4 Door. Good condition. Leve...,al
55,19900,2004.0,ford,f250 super duty,good,8 cylinders,full-size,pickup,blue,Here I have a unmolested Ford F-250 6.0 power ...,al
59,14000,2012.0,honda,odyssey,excellent,6 cylinders,full-size,mini-van,silver,"Readily available. Excellent condition, clean ...",al
65,22500,2001.0,ford,f450,good,8 cylinders,full-size,truck,white,"Ford Super Duty F450 , XL with King Ranch seat...",al
73,15000,2017.0,dodge,charger rt 4dr sedan,excellent,8 cylinders,mid-size,sedan,grey,"2017 dodge charger rt runs great $15,000.00",al


### 🔹 Encodage des variables catégoriques

In [ ]:
categorical_cols = ['manufacturer', 'model', 'condition', 'cylinders', 'size', 'type', 'paint_color', 'state']
encoder = OneHotEncoder(sparse_output=True, drop='first')
encoded_features = encoder.fit_transform(df[categorical_cols])
df_encoded = pd.DataFrame.sparse.from_spmatrix(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = pd.concat([df.drop(columns=categorical_cols), df_encoded], axis=1)


NameError: name 'df' is not defined

### 🔹 Normalisation des features numériques

In [ ]:
scaler = StandardScaler()
df[['year']] = scaler.fit_transform(df[['year']])
df.head()

## 2️⃣ Entraînement du modèle de prédiction

In [ ]:
X = df.drop(columns=['price', 'description'])
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

## 3️⃣ Évaluation du modèle

In [ ]:
y_pred = model.predict(X_test)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred.flatten())
plt.xlabel('Prix réel')
plt.ylabel('Prix prédit')
plt.title('Comparaison entre prix réel et prédit')
plt.show()

## 4️⃣ Sauvegarde du modèle pour API

In [ ]:
model.save('car_price_model.h5')
print('Modèle sauvegardé ✅')

## 5️⃣ Génération d’images avec Stable Diffusion

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained('CompVis/stable-diffusion-v1-4')
pipe.to('cuda')

def generate_car_image(description, predicted_price):
    prompt = f'Voiture {description}, estimée à {int(predicted_price)}€, design moderne et détaillé'
    image = pipe(prompt).images[0]
    image.show()
    image.save('generated_car.png')

# Exemple d'utilisation
predicted_price = model.predict(X_test.iloc[0:1])[0][0]
description = df.iloc[0]['description']
generate_car_image(description, predicted_price)